# Build Agentic Workflows with Hugging Face Smolagents in Snowflake

## Overview

This guide outlines the process for creating agentic workflows in Snowflake Notebook on Container Runtime using [Smolagents from Hugging Face](https://github.com/huggingface/smolagents). These agents are capable of writing Python code to call tools and orchestrate other agents. In this guide, we will also see how you can create a custom tool in **Smolagent** that uses Snowflake Cortex.

## Step-By-Step Guide

For prerequisites and environment setup, please refer to the [QuickStart Guide](https://quickstarts.snowflake.com/guide/build-agentic-workflows-with-huggingface-smolagents-in-snowflake/index.html).

In [ ]:
# https://github.com/huggingface/smolagents

!pip install smolagents

In [ ]:
import streamlit as st
import pandas as pd
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, tool, LiteLLMModel, ToolCollection, Tool, ToolCallingAgent
import requests
from bs4 import BeautifulSoup

# TODO: Replace hf_ZkEXVwIXXXXXXXXXXXXXXX with your Hugging Face token
model = HfApiModel(token="hf_ZkEXVwIXXXXXXXXXXXXXXX", provider='hf-inference')

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10,
    name='web_search',
    description="Runs web searches for you. Give it your query as an argument.",
)

code_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=['requests','json','pandas','matplotlib','xml.etree.ElementTree','bs4'],
)

In [ ]:
answer = 'N/A'
try:
    answer = code_agent.run("""
        Top 5 announcements at Snowflake Summit 2024 in JSON format. 
        Only return the JSON formatted output as the response and nothing else.
    """)
    parsed_answer = json.loads(answer)
    st.json(parsed_answer)
except:
    st.write(answer)

In [ ]:
answer = code_agent.run("""
        Top 5 blog articles on AI. Include blog title and link to the article. 
        Return the response in a Pandas dataframe and nothing else.
    """)
try:
    st.dataframe(pd.DataFrame(answer))
except:
    st.write(answer)

In [ ]:
@tool
def get_dzone_refcards() -> str:
    """
    This tool returns a list of popular DZone Refcards.
    """
    try:
      url = "https://dzone.com/refcardz?sort=popular"
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      soup = BeautifulSoup(response.content, "html.parser")
      refcard_containers = soup.find_all('div', class_='pin')
      refcard_links = []
      for refcard_container in refcard_containers:
          refcard_subtitle = refcard_container.find('div', class_='asset-subtitle')
          refcard_link = refcard_subtitle.find('a')['href']
          refcard_links.append(refcard_link)
    except requests.exceptions.RequestException as e:
      print(f"Error occurred while fetching the HTML: {e}")
        
    return refcard_links

@tool
def read_dzone_refcards(link: str) -> str:
    """
    This tool reads a DZone Refcard and returns the HTML text.
    Args:
        link: Link of the RefCard to read.
    """
    refcard_text = ""
    try:
      refcard_url = f"https://dzone.com/{link}"
      response = requests.get(refcard_url)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      refcard_text = BeautifulSoup(response.content, "html.parser").text
    except requests.exceptions.RequestException as e:
      print(f"Error occurred while fetching the HTML: {e}")
        
    return refcard_text

@tool
def summarize_article(article_text: str) -> any:
    """
    This tool summarizes article text using Snowflake Cortex AI.
    Args:
        article_text: Text to summarize.
    """
    import snowflake
    from snowflake.cortex import Complete
    summary = ""
    try:
        prompt = f"""
                Summarize the text enclosed in ### in less than 500 words: ### {article_text} ###.
                Produce JSON output that includes article title, article url, article summary, and 3 highlights from the article.
        """
        summary = snowflake.cortex.Complete('claude-3-5-sonnet',prompt)
    except Exception as e:
      print(f"Error occurred: {e}")
        
    return summary

In [ ]:
agent = CodeAgent(tools=[get_dzone_refcards,
                         read_dzone_refcards,
                         summarize_article,],
                  model=model,
                  max_steps=20,
                  add_base_tools=True)
try:
    st.write(agent.run("Generate a list of popular DZone RefCard summaries by reading them.",))
except Exception as e:
    print(f"Error occurred: {e}")